### DataParallel
    接下来介绍，在多GPU环境下，如何出了数据并行(DataParallel)。
    其实，pytorch中实现DataParallel很简单

In [1]:
import torch
device = torch.device('cuda:0')
model.to(device) #将模型拷贝到GPU上

NameError: name 'torch' is not defined

然后，我们可以把tensor都拷贝到GPU上

In [ ]:
mytensor = my_tensor.to(device)

mytensor是一个新的拷贝，存放在GPU上。
上面的代码，已经可以实现在GPU上运行神经网络。但是在多个GPU环境中，由于pytorch默认只会使用一个GPU，要使用多GPU的数据并行处理，pytorch使用DataParallel完成。

In [ ]:
import torch.nn as nn

model = nn.DataParallel(model)

#### import and parameter

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#parameters and loader
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

#### Devices

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#### Dummy Dataset
    生成随机数据集

In [4]:
class RandomDataset(Dataset):

    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len

rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size),
                         batch_size=batch_size, shuffle=True)

定义简单模型(神经网络)

In [5]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, input):
        output = self.fc(input)
        print("\tIn Model: input size", input.size(),
              "output size", output.size())
        return output

#### 创建模型和DataParallel
    首先判断设备中GPU个数，如果在多GPU环境下，需要将模型放入DataParallel中，并将模型放入GPU。

In [6]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)

model.to(device)

TypeError: __init__() takes 1 positional argument but 3 were given

#### 运行模型

In [ ]:
for data in rand_loader:
    input = data.to(device)  #将数据放入cuda设备中
    output = model(input)  #模型已经在cuda设备中
    print("Outside: input size", input.size(),
          "output_size", output.size())

如果没有GPU设备或者只有一个GPU设备，如果输入的input_size=30,output_size = 30,则输出为30和output_size = 30.
如果有多个GPU设备的话，数据batch会将batch_size分配到各个GPU上。
例如，在input_size = 30, output_size = 30的情况下，2个GPU的话，每个GPU分配15个batch；如果是3个GPU，每个GPU分配10个batch，8个GPU的话，每个GPU分配4个batch

#### 总结
    DataParallel将数据自动分配到各个GPU中，并给各个GPU上的模型发送任务。当每个GPU上的模型完成任务后，DataParallel合并结果，返回给用户。